In [1]:
#Program Name: parameters.ipynb
#Author: Deepesh B. Verma (dbv293) 

In [2]:
import re 

In [3]:
class Parameters: 
    """
    A class to read the parameters file of GENE to assign variables for the siimulation.
    Scrapes values from a parameters_XXXX GENE file. 
    """
    #List of variables to be read from the parameters file. 
    spec_nl=('omn','omt','mass','charge','dens','temp','name','passive','kappa_n','kappa_T','LT_center','Ln_center',
             'LT_width','Ln_width','prof_type','src_prof_type','src_amp','src_width','src_x0','delta_x_n','delta_x_T')

    def __init__(self): pass 

    def clearcomments(self,variable):
        """ 
        Clears comments from line in the parameters file so that a parameter value can be assigned.
        """
        regex=re.compile(r'\s*([-+\'\"\[\];.,/a-zA-Z0-9_\s*]*)\s*!?\s*(.*)')
        result=regex.search(variable)
        if result and result.group(2)[:4]!='scan': 
            return regex.search(variable).group(1)
        else:
            return variable
    
    def Read_Pars(self,path): 
        """
        Reads the GENE parameters file and assigns those values to variables.
        """
        #Opens the parameters file. 
        parfile=open(path,"r")

        #Dictionary for the parameters {parameter: value}.
        self.pardict={}
        #Dictionary for recording the namelists that the parameters belong to: {parameter: namelist}
        self.nmldict={}
        #Keeps track of all namelists that have been found.
        self.namelists=[]
        #Counts species namelists
        countspec=0

        #Searches parameters_XXXX file line by line for parameters using regular expressions.
        for line in parfile:

            #Excludes commented lines in the parameters file. 
            if re.search(r'\s*!\w*\s*=.*',line)==None:

                #Checks for species namelists and counts the number of species. 
                if re.search(r'^\s*&(.*)',line):
                    #If namelist belongs to a species, append its number to the namelist.
                    if re.search(r'^\s*&(.*)',line).group(1)=='species':
                        countspec+=1
                        nml=re.search(r'^\s*&(.*)',line).group(1)+str(countspec)
                    else: nml=re.search(r'^\s*&(.*)',line).group(1)
                    if nml not in self.namelists: self.namelists.append(nml)

                #Search lines for a parameter and its corresponding value.
                p=re.compile(r'^\s*(.*)\s*=\s*(.*)')
                m=p.search(line)
                #Pick matching lines and build parameter dictionary.
                if m:
                    if m.group(1).strip() in self.spec_nl:
                        self.pardict[m.group(1).strip()+str(countspec)]=m.group(2)
                        self.nmldict[m.group(1).strip()+str(countspec)]=nml
                    else:
                        self.pardict[m.group(1).strip()]=m.group(2)
                        self.nmldict[m.group(1).strip()]=nml
                
        #Clears comments from all variables.
        for item in self.pardict:
            self.pardict[item]=self.clearcomments(self.pardict[item])

In [4]:

def init_read_parameters_file(suffix):
    """
    Imports parameters and assign values from GENE file for parameters_XXXX.
    Suffix is the run XXXX.
    """

    par = Parameters()
    par.Read_Pars('parameters'+suffix)
    pars = par.pardict
    return pars

def read_ref_values(suffix, pars):
    """
    Assigns references values from pars. Assigns 1.0 if not specified.
    """

    #Magnetic field reference value.
    if 'Bref' in pars:
        Bref = pars['Bref']
    else:
        Bref = 1.
        print('Bref not in parameters'+ suffix + '. Bref = 1')

    #Temperature reference value.
    if 'Tref' in pars:
        Tref = pars['Tref']
    else:
        Tref = 1.
        print('Tref not in parameters'+ suffix + '. Tref = 1')

    #Density reference value.
    if 'nref' in pars:
        nref = pars['nref']
    else:
        nref = 1.
        print('nref not in parameters'+ suffix + '. nref = 1')

    #Length reference value.
    if 'Lref' in pars:
        Lref = pars['Lref']
    else:
        Lref = 1.
        print('Lref not in parameters'+ suffix + '. Lref = 1')

    #Mass reference value. 
    if 'mref' in pars:
        mref = pars['mref']
    else:
        mref = 1.
        print('mref not in parameters'+ suffix + '. mref = 1')
    
    return Bref, Tref, nref, Lref, mref